In [1]:
#Improved Multi-objective Particle swarm Optimization Algorithm Implementation

import numpy as np

# Generate population
def generate_population(bounds, n_vars, N):
    return np.random.uniform(bounds[:, 0], bounds[:, 1], size=(N, n_vars))

# Define the IMOPSO algorithm
def imopso(objective_function, n_vars, population, r1, r2, c1, c2, w, Tmax, Amax):
    # Initialization
    N = len(population)
    particles_pos = population.copy()
    particles_vel = np.zeros_like(particles_pos)  # Initialize particles_vel with the same shape as particles_pos
    p_best = particles_pos.copy()
    p_best_fitness = np.array([objective_function(p) for p in p_best])
    external_archive = p_best.copy()
    external_archive_fitness = p_best_fitness.copy()

    # Main loop
    for T in range(1, Tmax+1):
        for i in range(N):
            # Update velocity
            rp = np.random.rand()
            rg = np.random.rand()
            particles_vel[i] = w * particles_vel[i] + \
                               c1 * rp * (p_best[i] - particles_pos[i]) + \
                               c2 * rg * (external_archive[np.random.randint(len(external_archive))] - particles_pos[i]) #Velocity Update
            # Update position
            particles_pos[i] += particles_vel[i]    #Position Update
            particles_pos[i] = np.clip(particles_pos[i], bounds[:, 0], bounds[:, 1])

            # Update p_best and external_archive
            fitness = objective_function(particles_pos[i])
            if np.all(fitness <= p_best_fitness[i]):
                p_best[i] = particles_pos[i]
                p_best_fitness[i] = fitness
                if np.all(fitness <= external_archive_fitness):
                    external_archive = np.vstack((external_archive, particles_pos[i]))
                    external_archive_fitness = np.vstack((external_archive_fitness, fitness))

        # Truncate external archive if necessary
        if len(external_archive) > Amax:
            external_archive, external_archive_fitness = truncate_archive(external_archive, external_archive_fitness, Amax)



    return external_archive, external_archive_fitness

# Helper function to calculate crowding distance
def calculate_crowding_distance(archive_fitness):
    num_objectives = archive_fitness.shape[1]
    num_solutions = len(archive_fitness)
    crowding_distance = np.zeros(num_solutions)

    for i in range(num_objectives):
        sorted_indices = np.argsort(archive_fitness[:, i])
        crowding_distance[sorted_indices[0]] = np.inf
        crowding_distance[sorted_indices[-1]] = np.inf
        max_val = np.max(archive_fitness[:, i])
        min_val = np.min(archive_fitness[:, i])
        if max_val == min_val:
            continue
        normalized_fitness = (archive_fitness[:, i] - min_val) / (max_val - min_val)
        for j in range(1, num_solutions - 1):
            crowding_distance[sorted_indices[j]] += normalized_fitness[sorted_indices[j + 1]] - normalized_fitness[sorted_indices[j - 1]]

    return crowding_distance

# Helper function to truncate external archive
def truncate_archive(archive, archive_fitness, Amax):
    crowding_distance = calculate_crowding_distance(archive_fitness)
    sorted_indices = np.argsort(crowding_distance)
    archive = archive[sorted_indices[:Amax]]
    archive_fitness = archive_fitness[sorted_indices[:Amax]]
    return archive, archive_fitness

# Objective functions
def SCH(x):
    return [x[0]**2, (x[0] - 2)**2]

def FON(x):
    return [1 - np.exp(-np.sum((x - np.sqrt(1/3))**2)), 1 - np.exp(-np.sum((x + np.sqrt(1/3))**2))]

def KUR(x):
    return [np.sum(-10 * np.exp(-0.2 * np.sqrt(np.maximum(0, x[1:])))), np.sum(np.abs(x)**0.8 + 5 * np.sin(x**3))]


def ZDT1(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.maximum(0, 1 - (f1 / g)**2)))  # Ensure non-negative argument for square root
    return [f1, f2]

def ZDT2(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * np.maximum(0, 1 - (f1 / g) ** 2)  # Ensure non-negative value
    return [f1, f2]




def ZDT3(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.maximum(0, 1 - (f1 / g)**2)) - (f1 / g) * np.sin(10 * np.pi * f1))
    return [f1, f2]

def ZDT4(x):
    f1 = x[0]
    g = 1 + 10 * (len(x) - 1) + np.sum(x[1:]**2 - 10 * np.cos(4 * np.pi * x[1:]))
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.where(g != 0, f1 / g, 0)))
    return [f1, f2]

def ZDT6(x):
    f1 = 1 - np.exp(-4 * x[0]) * np.sin(6 * np.pi * x[0])**6
    g = np.sum(x[1:]) / (len(x) - 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero

    if g == 0:
        f2 = 1
    else:
        f2 = (1 - (f1 / g)**2)

    return [f1, f2]




# Define objective functions, dimensions, bounds, and other parameters
objective_functions = [SCH, FON, KUR, ZDT1, ZDT2, ZDT3, ZDT4, ZDT6]
n_vars_list = [1, 3, 3, 30, 30, 30, 10, 10]  # Number of variables for each objective function
bounds_list = [np.array([[-1000, 1000]]),  # Bounds for SCH
               np.array([[-4, 4], [-4, 4], [-4, 4]]),  # Bounds for FON
               np.array([[-5, 5], [-5, 5], [-5, 5]]),  # Bounds for KUR
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT1
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT2
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT3
               np.array([[0, 1] for _ in range(10)]),  # Bounds for ZDT4
               np.array([[0, 1] for _ in range(10)])]  # Bounds for ZDT6

r1, r2, c1, c2, w, Tmax, Amax = 0.5, 0.5,1.49618, 1.49618, 0.72984, 1000, 100
# Generate fixed population
population = generate_population(bounds_list[0], n_vars_list[0], N=10)

# Run IMOPSO for each objective function and generate statistics
statistics = []
for i, objective_function in enumerate(objective_functions):
    bounds = bounds_list[i]  # Fetch bounds for the current objective function
    population = generate_population(bounds, n_vars_list[i], N=10)  # Generate population based on bounds
    external_archive, external_archive_fitness = imopso(objective_function, n_vars_list[i], population, r1, r2, c1, c2, w, Tmax, Amax)
    max_val, min_val, avg_val, std_val = np.nanmax(external_archive_fitness), np.nanmin(external_archive_fitness), np.nanmean(external_archive_fitness), np.nanstd(external_archive_fitness)

    statistics.append([objective_function.__name__, max_val, min_val, avg_val, std_val])

# Print table
header = ["Objective function", "Max", "Min", "Average", "Std"]
print("{:<20} {:<20} {:<20} {:<20} {:<20}".format(*header))
for row in statistics:
    print("{:<20} {:<20.4e} {:<20.4e} {:<20.4e} {:<20.4e}".format(*row))


Objective function   Max                  Min                  Average              Std                 
SCH                  9.0433e+05           9.7832e-01           2.6873e+05           3.4596e+05          
FON                  1.0000e+00           2.7311e-01           9.1249e-01           1.9296e-01          
KUR                  1.6716e+01           -2.0000e+01          -9.7598e+00          1.1135e+01          
ZDT1                 0.0000e+00           0.0000e+00           0.0000e+00           0.0000e+00          
ZDT2                 1.9310e+00           0.0000e+00           9.6552e-01           9.6552e-01          
ZDT3                 9.8811e-01           -4.2392e-01          3.2337e-01           3.6262e-01          
ZDT4                 5.0000e+00           0.0000e+00           2.5000e+00           2.5000e+00          
ZDT6                 1.0000e+00           -6.6040e+00          -1.1195e+00          2.5313e+00          
